In [1]:
from bs4 import BeautifulSoup
import requests
import html5lib
import re
import pandas as pd
import numpy as np
# pip install pyopenssl ndg-httpsclient pyasn1

In [3]:
cookies = {
    'ccsid': '668-8707266-8901325',
    '__qca': 'P0-1002348396-1661184067604',
    'p': 'ZqHh-ArV88dYkVHfgapFDgVJRXU5lLzNc4VfzdNPfI5ZMVgd',
    'likely_has_account': 'true',
    'srb_8': '0_wl',
    'locale': 'en',
    'u': 'oTJu21ixofG3HvrbsDKN1hg8e8FfKW_FUT1R8ZnFKULdyBce',
    'csm-sid': '696-1026276-2871101',
    'logged_out_browsing_page_count': '2',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.9,id;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    # 'Cookie': 'ccsid=668-8707266-8901325; __qca=P0-1002348396-1661184067604; p=ZqHh-ArV88dYkVHfgapFDgVJRXU5lLzNc4VfzdNPfI5ZMVgd; likely_has_account=true; srb_8=0_wl; locale=en; u=oTJu21ixofG3HvrbsDKN1hg8e8FfKW_FUT1R8ZnFKULdyBce; csm-sid=696-1026276-2871101; logged_out_browsing_page_count=2',
    'If-None-Match': 'W/"4279d1cc80c6f7b6843a18660d55f9f2"',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not?A_Brand";v="8", "Chromium";v="108", "Google Chrome";v="108"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}


In [4]:
cookies2 = {
    'ccsid': '668-8707266-8901325',
    '__qca': 'P0-1002348396-1661184067604',
    'p': 'ZqHh-ArV88dYkVHfgapFDgVJRXU5lLzNc4VfzdNPfI5ZMVgd',
    'likely_has_account': 'true',
    'srb_8': '0_wl',
    'locale': 'en',
    'u': 'oTJu21ixofG3HvrbsDKN1hg8e8FfKW_FUT1R8ZnFKULdyBce',
    'csm-sid': '696-1026276-2871101',
    'logged_out_browsing_page_count': '2',
    '_session_id2': 'ae3b31e88be4ac3d2df2cc4e36100b58',
}

headers2 = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.9,id;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    # 'Cookie': 'ccsid=668-8707266-8901325; __qca=P0-1002348396-1661184067604; p=ZqHh-ArV88dYkVHfgapFDgVJRXU5lLzNc4VfzdNPfI5ZMVgd; likely_has_account=true; srb_8=0_wl; locale=en; u=oTJu21ixofG3HvrbsDKN1hg8e8FfKW_FUT1R8ZnFKULdyBce; csm-sid=696-1026276-2871101; logged_out_browsing_page_count=2; _session_id2=ae3b31e88be4ac3d2df2cc4e36100b58',
    'If-None-Match': 'W/"b850254e01551ab34564b673d5620452"',
    'Referer': 'https://www.goodreads.com/shelf/show/to-read?page=1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not?A_Brand";v="8", "Chromium";v="108", "Google Chrome";v="108"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

In [5]:
def clean_publish_year(raw_year):
    if type(raw_year) is None:
        return "Missing"
    result = re.search(r"(\b\d+)$", raw_year)
    if result == None:
        return 0
    else:
        return result.group(1)

def clean_rating(raw_rating):
    if type(raw_rating) is None:
        return 0
    result = re.search(r"(\b\d+.\b\d+)", raw_rating)
    if result == None:
        return 0
    else:
        return result.group(1)

def clean_raters(raw_raters):
    if type(raw_raters) is None:
        return 0
    result = re.search(r"((?:\d+,?\d+)*) ratings", raw_raters)
    if result == None:
        return 0
    else:
        return result.group(1)
    
def clean_genres(raw_genres):
    genreList = []
    if raw_genres == None:
        return "Missing"
    else:
        for genre in raw_genres:
            if  genre == None:
                continue
            else:
                genreList.append(genre.find("a").get_text())
        return genreList
# def clean_readers(raw_readers):
#     result = re.search(r"(\b\d+)", raw_readers)
#     if result == None:
#         return "None"
#     else:
#         return result.group(1)

def solve_nonetype_error(value):
    if value is not None:
        return value.text.strip()
    else:
        return "Missing"



In [7]:

books = []
for page in range(1, 26):
    response = requests.get("https://www.goodreads.com/shelf/show/to-read?page=" + str(page), cookies = cookies, headers=headers)
    html_soup = BeautifulSoup(response.text, 'html5lib')
    book_container = html_soup.find_all('div', class_ = "elementList")
    print("this is page " , page)
    for book in book_container:
        bookTitle = solve_nonetype_error(book.find('a', class_ = "bookTitle"))
       
        bookAuthor =solve_nonetype_error(book.find('a', class_ = "authorName"))

        # After(use comments)
        year = clean_publish_year(solve_nonetype_error(book.find('span', class_ = "greyText smallText")))
        rating = clean_rating(solve_nonetype_error(book.find('span', class_ = "greyText smallText")))
        raters = str(clean_raters(solve_nonetype_error(book.find('span', class_ = "greyText smallText")))).replace(",", "")
        int(raters)
        # # readerNum = clean_readers(book.find('span', class_ = "greyText statistic").text.strip())
        
        # Before (use comments)
        # year = solve_nonetype_error(book.find('span', class_ = "greyText smallText"))
        # rating = solve_nonetype_error(book.find('span', class_ = "greyText smallText"))
        # raters = solve_nonetype_error(book.find('span', class_ = "greyText smallText"))
        # readerNum = book.find('span', class_ = "greyText statistic").text.strip()
        print(bookTitle)
        link = book.a['href']
        completeLink = "https://www.goodreads.com" + link
        # getGenre = requests.get(completeLink)
        getGenre = requests.get(completeLink,  cookies = cookies2, headers=headers2)
        genreParsing = BeautifulSoup(getGenre.text, 'html5lib')
        genres = clean_genres(genreParsing.findAll("div", class_=  "elementList", style = None))
        genreStr = ",".join(genres)
        book_data = [bookTitle, bookAuthor, year, rating, raters, genreStr]
        books.append(book_data)
books = pd.DataFrame(books, columns=["Title", "Author", "Year", "Rating", "Raters", "Genres"])
books.head()

this is page  1
To Kill a Mockingbird (Paperback)
1984 (Paperback)
Pride and Prejudice (Paperback)
The Book Thief (Hardcover)
The Fault in Our Stars (Hardcover)
The Diary of a Young Girl (Mass Market Paperback)
The Great Gatsby (Paperback)
The Alchemist (Paperback)
All the Light We Cannot See (Hardcover)
The Seven Husbands of Evelyn Hugo (Hardcover)
Harry Potter and the Philosopher's Stone (Harry Potter, #1)
The Hunger Games (The Hunger Games, #1)
The Girl with the Dragon Tattoo (Millennium, #1)
The Girl on the Train (Hardcover)
Steve Jobs (Hardcover)
The Catcher in the Rye (Paperback)
Fahrenheit 451 (Kindle Edition)
The Handmaid's Tale (The Handmaid's Tale, #1)
The Midnight Library (Hardcover)
Jane Eyre (Paperback)
Twilight (The Twilight Saga, #1)
Little Women (Paperback)
Gone Girl (Paperback)
The Kite Runner (Paperback)
Wuthering Heights (Paperback)
Where the Crawdads Sing (ebook)
The Invisible Life of Addie LaRue (Hardcover)
Divergent (Divergent, #1)
Sapiens: A Brief History of Huma

,Title,Author,Year,Rating,Raters,Genres
0,To Kill a Mockingbird (Paperback),Harper Lee,1960,4.27,5490833,"Classics,Fiction,Historical,Academic,Literatur..."
1,1984 (Paperback),George Orwell,1949,4.19,4016657,"Classics,Fiction,Science Fiction,Science Ficti..."
2,Pride and Prejudice (Paperback),Jane Austen,1813,4.28,3785713,"Classics,Fiction,Romance,Historical,Literature..."
3,The Book Thief (Hardcover),Markus Zusak,2006,4.39,2274375,"Historical,Fiction,Young Adult,Historical,Clas..."
4,The Fault in Our Stars (Hardcover),John Green,2012,4.16,4563756,"Young Adult,Romance,Fiction,Contemporary,Reali..."


In [8]:
books.to_csv("ToReadWithGenre.csv", index = False)


In [9]:
df = pd.read_csv("ToReadWithGenre.csv")

df = df[df.Author != "Missing"]

df.isnull().sum()

df.to_csv("ToReadWithGenreCleaned.csv")
